In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('SparkDataFrameDemo').getOrCreate()

22/05/23 16:27:51 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.108.128 instead (on interface ens33)
22/05/23 16:27:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/23 16:27:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
input_path = "Admission_Predict.csv"
df = spark.read.option('header', True).csv(input_path)
df.take(5)

[Row(Serial No.='1', GRE Score='337', TOEFL Score='118', University Rating='4', SOP='4.5', LOR ='4.5', CGPA='9.65', Research='1', Chance of Admit ='0.92'),
 Row(Serial No.='2', GRE Score='324', TOEFL Score='107', University Rating='4', SOP='4', LOR ='4.5', CGPA='8.87', Research='1', Chance of Admit ='0.76'),
 Row(Serial No.='3', GRE Score='316', TOEFL Score='104', University Rating='3', SOP='3', LOR ='3.5', CGPA='8', Research='1', Chance of Admit ='0.72'),
 Row(Serial No.='4', GRE Score='322', TOEFL Score='110', University Rating='3', SOP='3.5', LOR ='2.5', CGPA='8.67', Research='1', Chance of Admit ='0.8'),
 Row(Serial No.='5', GRE Score='314', TOEFL Score='103', University Rating='2', SOP='2', LOR ='3', CGPA='8.21', Research='0', Chance of Admit ='0.65')]

In [4]:
df.printSchema()

root
 |-- Serial No.: string (nullable = true)
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR : string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit : string (nullable = true)



In [5]:
df.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|         5|      314|        103|                2|  2|   3|8.21|       0|            0.65|
|         6|      330|        115|                5|4.5|   3|9.34|       1|             0.9|
|         7|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|         8|      308|        101|                2|  3|   4| 7.9|    

In [6]:
df.createOrReplaceTempView("Graduate")

In [7]:
df2 = spark.sql("SELECT * FROM Graduate WHERE Research=1")
df2.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|         6|      330|        115|                5|4.5|   3|9.34|       1|             0.9|
|         7|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|        11|      325|        106|                3|3.5|   4| 8.4|       1|            0.52|
|        12|      327|        111|                4|  4| 4.5|   9|    

In [8]:
# we can drop the temprory view
spark.catalog.dropTempView("Graduate")

In [9]:
from pyspark.sql.functions import exp, lit 
df_with_nc1 = df.withColumn("n1", lit(0))
df_with_nc2 = df_with_nc1.withColumn("n2", exp("CGPA"))

In [10]:
df_with_nc1.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+---+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit | n1|
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+---+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|  0|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|  0|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|  0|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|  0|
|         5|      314|        103|                2|  2|   3|8.21|       0|            0.65|  0|
|         6|      330|        115|                5|4.5|   3|9.34|       1|             0.9|  0|
|         7|      321|        109|                3|  3|   4| 8.2|       1|            0.75|  0|
|         8|      308|        

In [11]:
df_with_nc2.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+---+------------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit | n1|                n2|
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+---+------------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|  0|15521.788104196934|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|  0| 7115.280973169776|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|  0|2980.9579870417283|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|  0| 5825.499349524731|
|         5|      314|        103|                2|  2|   3|8.21|       0|            0.65|  0|3677.5424662662012|
|         6|      330|        115|                5|4.5|   3|9.34|      

In [12]:
#spark.stop()

## Aggregate multiple columms

In [13]:
from pyspark.sql.functions import udf, expr, col
from pyspark.sql.types import ArrayType, StringType, DoubleType, StructField, StructField

In [14]:
df = spark.sparkContext.parallelize([\
        ("mary", "lemon", 2.00),\
        ("adam", "grape", 1.22),\
        ("adam", "carrot", 2.44),\
        ("adam", "orange", 1.99),\
        ("john", "tomato", 1.99),\
        ("john", "carrot", 0.45),\
        ("john", "banana", 1.29),\
        ("bill", "apple", 0.99),\
        ("bill", "taco", 2.59)\
        ]).toDF(["name", "food", "price"])

In [15]:
df.show()

+----+------+-----+
|name|  food|price|
+----+------+-----+
|mary| lemon|  2.0|
|adam| grape| 1.22|
|adam|carrot| 2.44|
|adam|orange| 1.99|
|john|tomato| 1.99|
|john|carrot| 0.45|
|john|banana| 1.29|
|bill| apple| 0.99|
|bill|  taco| 2.59|
+----+------+-----+



In [16]:
df.groupBy('name').agg(expr('collect_list(food) as food'), expr('collect_list(price) as price')).show(truncate=False)

+----+------------------------+------------------+
|name|food                    |price             |
+----+------------------------+------------------+
|adam|[grape, carrot, orange] |[1.22, 2.44, 1.99]|
|mary|[lemon]                 |[2.0]             |
|john|[tomato, carrot, banana]|[1.99, 0.45, 1.29]|
|bill|[apple, taco]           |[0.99, 2.59]      |
+----+------------------------+------------------+



In [17]:
@udf("array<struct<_1:string, _2:double>>")
def zip_(xs, ys):
    return list(zip(xs, ys))
df.groupBy('name').agg(expr('collect_list(food) as food'), expr('collect_list(price) as price')).withColumn('x', zip_(col('food'), col('price'))).show(truncate=False)

+----+------------------------+------------------+------------------------------------------------+
|name|food                    |price             |x                                               |
+----+------------------------+------------------+------------------------------------------------+
|adam|[grape, carrot, orange] |[1.22, 2.44, 1.99]|[{grape, 1.22}, {carrot, 2.44}, {orange, 1.99}] |
|mary|[lemon]                 |[2.0]             |[{lemon, 2.0}]                                  |
|john|[tomato, carrot, banana]|[1.99, 0.45, 1.29]|[{tomato, 1.99}, {carrot, 0.45}, {banana, 1.29}]|
|bill|[apple, taco]           |[0.99, 2.59]      |[{apple, 0.99}, {taco, 2.59}]                   |
+----+------------------------+------------------+------------------------------------------------+



In [18]:
#We can also do operations in a pandas-like method
#However it seems that we can only apply one function to the grouped object
df.groupBy('name').agg({'price': 'max', 'price':'min', 'price': 'mean'}).show(truncate=False)

+----+------------------+
|name|avg(price)        |
+----+------------------+
|adam|1.8833333333333335|
|mary|2.0               |
|john|1.2433333333333334|
|bill|1.79              |
+----+------------------+



In [19]:
from pyspark.sql.functions import min, max, avg
funcs = [min, max, avg]
to_agg = ["price"]
exprs = [f(col(name)) for f in funcs for name in to_agg]
df.groupBy('name').agg(*exprs).show()

+----+----------+----------+------------------+
|name|min(price)|max(price)|        avg(price)|
+----+----------+----------+------------------+
|adam|      1.22|      2.44|1.8833333333333335|
|mary|       2.0|       2.0|               2.0|
|john|      0.45|      1.99|1.2433333333333334|
|bill|      0.99|      2.59|              1.79|
+----+----------+----------+------------------+



In [22]:
!pip install pyarrow

     |████████████████████████████████| 29.4 MB 4.5 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [25]:
import pandas as pd 
from pyspark.sql.functions import pandas_udf
@pandas_udf('string')
def upper(x: pd.Series) -> pd.Series:
    return x.str.upper()

df.groupBy('name').agg(upper(col('name'))).show()

+----+-----------+
|name|upper(name)|
+----+-----------+
|adam|       ADAM|
|mary|       MARY|
|john|       JOHN|
|bill|       BILL|
+----+-----------+



## Create a UDF

In [26]:
def squared(x):
    return x**2

squared_udf = udf(squared, DoubleType())
df.select('name', 'price', squared_udf(col('price')).alias('squared_price')).show()

+----+-----+------------------+
|name|price|     squared_price|
+----+-----+------------------+
|mary|  2.0|               4.0|
|adam| 1.22|            1.4884|
|adam| 2.44|            5.9536|
|adam| 1.99|            3.9601|
|john| 1.99|            3.9601|
|john| 0.45|            0.2025|
|john| 1.29|1.6641000000000001|
|bill| 0.99|            0.9801|
|bill| 2.59| 6.708099999999999|
+----+-----+------------------+



## Create DataFrame from Rows

In [29]:
from pyspark.sql import Row

dept1 = Row(id='100', name='Computer Science')
dept2 = Row(id='200', name='Mechanical Engineering')
dept3 = Row(id='300', name='Music')
dept4 = Row(id='400', name='Sports')
dept5 = Row(id='500', name='Biology')

# Create the Employees
Employee = Row("first_name", "last_name", "email", "salary")

employee1 = Employee('alex', 'smith', 'alex@berkeley.edu', 110000)
employee2 = Employee('jane', 'goldman', 'jane@stanford.edu', 120000)
employee3 = Employee('matei', None, 'matei@yahoo.com', 140000)
employee4 = Employee(None, 'eastwood', 'jimmy@berkeley.edu', 160000)
employee5 = Employee('betty', 'ford', 'betty@gmail.com', 130000)

department_with_employees_1 = Row(department=dept1, employees = [employee1, employee2, employee5])
department_with_employees_2 = Row(department=dept2, employees=[employee3, employee4])
department_with_employees_3 = Row(department=dept3, employees=[employee1, employee4])
department_with_employees_4 = Row(department=dept4, employees=[employee2, employee3])
department_with_employees_5 = Row(department=dept5, employees=[employee5])


print("dept1=", dept1)
print("employee2=", employee2)
print("department_with_employees_1", department_with_employees_1)

dept1= Row(id='100', name='Computer Science')
employee2= Row(first_name='jane', last_name='goldman', email='jane@stanford.edu', salary=120000)
department_with_employees_1 Row(department=Row(id='100', name='Computer Science'), employees=[Row(first_name='alex', last_name='smith', email='alex@berkeley.edu', salary=110000), Row(first_name='jane', last_name='goldman', email='jane@stanford.edu', salary=120000), Row(first_name='betty', last_name='ford', email='betty@gmail.com', salary=130000)])


In [31]:
departments_with_employees_seq_1 = [department_with_employees_1, department_with_employees_2, department_with_employees_5]
spark.createDataFrame(departments_with_employees_seq_1).show()

+--------------------+--------------------+
|          department|           employees|
+--------------------+--------------------+
|{100, Computer Sc...|[{alex, smith, al...|
|{200, Mechanical ...|[{matei, null, ma...|
|      {500, Biology}|[{betty, ford, be...|
+--------------------+--------------------+



In [32]:
spark.createDataFrame(departments_with_employees_seq_1).printSchema()

root
 |-- department: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- employees: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- salary: long (nullable = true)



## Order By

In [33]:
df.orderBy(df.name).show()

+----+------+-----+
|name|  food|price|
+----+------+-----+
|adam| grape| 1.22|
|adam|orange| 1.99|
|adam|carrot| 2.44|
|bill|  taco| 2.59|
|bill| apple| 0.99|
|john|carrot| 0.45|
|john|tomato| 1.99|
|john|banana| 1.29|
|mary| lemon|  2.0|
+----+------+-----+



In [34]:
df.orderBy(df['food']).show()

+----+------+-----+
|name|  food|price|
+----+------+-----+
|bill| apple| 0.99|
|john|banana| 1.29|
|john|carrot| 0.45|
|adam|carrot| 2.44|
|adam| grape| 1.22|
|mary| lemon|  2.0|
|adam|orange| 1.99|
|bill|  taco| 2.59|
|john|tomato| 1.99|
+----+------+-----+



In [35]:
df.orderBy(df.price.desc()).show()

+----+------+-----+
|name|  food|price|
+----+------+-----+
|bill|  taco| 2.59|
|adam|carrot| 2.44|
|mary| lemon|  2.0|
|john|tomato| 1.99|
|adam|orange| 1.99|
|john|banana| 1.29|
|adam| grape| 1.22|
|bill| apple| 0.99|
|john|carrot| 0.45|
+----+------+-----+



In [36]:
df.summary().show()

+-------+----+------+------------------+
|summary|name|  food|             price|
+-------+----+------+------------------+
|  count|   9|     9|                 9|
|   mean|null|  null|1.6622222222222223|
| stddev|null|  null|0.7115261375694109|
|    min|adam| apple|              0.45|
|    25%|null|  null|              1.22|
|    50%|null|  null|              1.99|
|    75%|null|  null|               2.0|
|    max|mary|tomato|              2.59|
+-------+----+------+------------------+



In [40]:
median = df.approxQuantile("price", [0.02, 0.04], 0.01)
print("median:", median)

median: [0.45, 0.45]


## Multiple dims Aggregation

In [41]:
data = [\
        ('Ames', 2006, 100),\
        ('Ames', 2007, 200),\
        ('Ames', 2008, 300),\
        ('Sunnyvale', 2007, 10),\
        ('Sunnyvale', 2008, 20),\
        ('Sunnyvale', 2009, 30),\
        ('Stanford', 2008, 90)\
    ]

In [42]:
columns = ['city', 'year', 'amount']
sales = spark.createDataFrame(data, columns)
sales.show()

+---------+----+------+
|     city|year|amount|
+---------+----+------+
|     Ames|2006|   100|
|     Ames|2007|   200|
|     Ames|2008|   300|
|Sunnyvale|2007|    10|
|Sunnyvale|2008|    20|
|Sunnyvale|2009|    30|
| Stanford|2008|    90|
+---------+----+------+



In [44]:
groupby_city_and_year = sales.groupBy(['city', 'year']).agg(sum('amount')).alias('amount')
groupby_city_and_year.show()

## maybe because of city is string and year is int 

TypeError: unsupported operand type(s) for +: 'int' and 'str'